In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sympy
from sympy import Symbol as sb

In [36]:
RANDOM_STATE = 42

# Importing data

This is the data used for the regression experiments in the VRCP paper. It is generated by the PettingZoo Multi-Particle Environment (MPE) library (Terry et al. (2021)) for deep reinforcement learning using some specified parameters.

In [24]:
base_folder = 'data/mpe-data/spread/obs/example'
seed_dirs = [
    d for d in os.listdir(base_folder)
    if d.startswith("seed_") and os.path.isdir(os.path.join(base_folder, d))
]

initials_dict = {}
rewards_sum_dict = {}

for d in seed_dirs:
    seed_id = d.split('_', 1)[1]  # get the unique seed identifier
    folder = os.path.join(base_folder, d)
    state_file = os.path.join(folder, 'state_initial.npy')
    rewards_file = os.path.join(folder, 'rewards_traj_0.npy')
    
    if os.path.exists(state_file) and os.path.exists(rewards_file):
        # Load state_initial.npy
        state_initial = np.load(state_file, allow_pickle=True)
        initials_dict[seed_id] = state_initial
        
        # Load rewards_traj_0.npy and sum over column 0
        rewards_traj = np.load(rewards_file, allow_pickle=True)
        rewards_sum = rewards_traj[:, 0].sum()
        rewards_sum_dict[seed_id] = rewards_sum

# Create dataframes with the seed identifier as index
X = pd.DataFrame.from_dict(initials_dict, orient='index')
y = pd.DataFrame.from_dict(rewards_sum_dict, orient='index', columns=['reward_sum'])

In [27]:
X.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
6243,-4.638947,2.580580,-0.028328,0.248217,-3.924162,-4.063509,-0.416743,0.072911,-0.301039,-3.113701,-0.252470,-0.195096,-5.813191,3.822884,-5.418576,0.906666,-6.299335,4.016806
31994,4.281446,-3.261618,0.010150,-0.188225,4.297802,4.208184,0.092497,-0.479421,4.304405,1.721131,0.436334,0.264733,4.589279,-3.624757,2.610501,-2.089010,2.058335,-2.271987
35508,3.031222,1.169525,-0.074642,0.332341,-2.728792,4.987189,-0.359577,0.304756,-1.631607,-1.649202,-0.002981,0.405769,2.303867,1.535052,4.059220,0.225018,1.071398,1.865327
90144,-1.374741,0.999021,-0.418587,0.021922,1.123773,-2.763672,-0.340124,0.335538,-1.485562,-2.459087,-0.289571,0.358079,-0.304709,2.579039,-3.294965,-1.250610,-1.774850,1.165915
23835,-0.060958,-1.434050,-0.214032,0.328670,3.629799,4.153177,0.427419,0.381438,-3.286709,4.002148,0.473706,-0.422327,-0.989097,0.159420,2.319946,-1.123382,-0.530721,-0.815355


In [28]:
y.head(5)

,reward_sum
6243,-14.696764
31994,-13.417332
35508,-10.716178
90144,-12.103149
23835,-11.703057


## Train-test-split

In [46]:
test_size = 2000
calib_size = 2000

In [47]:
from sklearn.model_selection import train_test_split

# Split the data into training/calibration and testing sets
X_train_calib, X_test, y_train_calib, y_test = train_test_split(X, y, test_size=test_size, random_state=RANDOM_STATE)

# Split the training/calibration set into training and calibration sets
X_train, X_calib, y_train, y_calib = train_test_split(X_train_calib, y_train_calib, test_size=calib_size, random_state=RANDOM_STATE)

# Injecting ranges (symbolic expr)

In [50]:
from scipy.spatial import ConvexHull
import itertools

# useful functions

symbol_id = -1
def create_symbol(suffix=''):
    global symbol_id
    symbol_id += 1
    name = f'e{symbol_id}_{suffix}' if suffix else f'e{symbol_id}'
    return sympy.Symbol(name=name)


scaler_symbols = set([sb(f'k{i}') for i in range(X_calib.shape[1]+1)])
linearization_dict = dict()
reverse_linearization_dict = dict()


def sample_data(imputed_datasets, uncert_inds=[], seed=42):
    imp_np = np.array(imputed_datasets)
    if len(uncert_inds) == 0:
        uncert_inds = list(itertools.product(range(imp_np.shape[1]),range(imp_np.shape[2])))
    np.random.seed(seed)
    choices = np.random.choice(np.arange(imp_np.shape[0]), len(uncert_inds), replace=True)
    sample_result = imputed_datasets[0].copy()
    for i, ind in enumerate(uncert_inds):
        sample_result[ind[0]][ind[1]] = imputed_datasets[choices[i]][ind[0]][ind[1]]
    return sample_result


def linearization(expr_ls):
    processed_expr_ls = [0 for _ in range(len(expr_ls))]
    for expr_id, expr in enumerate(expr_ls):
        # Do not support monomial expr currently, e.g., expr = 1.5*e1. 
        # At lease two monomials in expr, e.g., expr = 1.5*e1 + 2.
        if not(expr.free_symbols):
            processed_expr_ls[expr_id] += expr
            continue
        expr = expr.expand()
        for arg in expr.args:
            if not(arg.free_symbols):
                processed_expr_ls[expr_id] += arg
                continue
            p = arg.as_poly()
            monomial_exponents = p.monoms()[0]
            
            # only deal with non-linear monomials (order > 2)
            if sum(monomial_exponents) <= 1:
                processed_expr_ls[expr_id] += arg
                continue

            monomial = sympy.prod(x**k for x, k in zip(p.gens, monomial_exponents) 
                                  if not(x in scaler_symbols))
            # check global substitution dictionary
            if monomial in linearization_dict:
                processed_expr_ls[expr_id] += arg.coeff(monomial)*linearization_dict[monomial]
            else:
                found = False
                subs_monomial = create_symbol()
                for symb in monomial.free_symbols:
                    if symb in reverse_linearization_dict:
                        equivalent_monomial = monomial.subs(symb, reverse_linearization_dict[symb])
                        if equivalent_monomial in linearization_dict:
                            subs_monomial = linearization_dict[equivalent_monomial]
                            found = True
                            break
                linearization_dict[monomial] = subs_monomial
                if not(found):
                    reverse_linearization_dict[subs_monomial] = monomial
                processed_expr_ls[expr_id] += arg.coeff(monomial)*subs_monomial
                
    return processed_expr_ls


def merge_small_components_pca(expr_ls, budget=10):
    if not(isinstance(expr_ls, sympy.Expr)):
        expr_ls = sympy.Matrix(expr_ls)
    if expr_ls.free_symbols:
        center = expr_ls.subs(dict([(symb, 0) for symb in expr_ls.free_symbols]))
    else:
        return expr_ls
    monomials_dict = get_generators(expr_ls)
    generators = np.array([monomials_dict[m] for m in monomials_dict])
    if len(generators) <= budget:
        return expr_ls
    monomials = [m for m in monomials_dict]
    pca = PCA(n_components=len(generators[0]))
    pca.fit(np.concatenate([generators, -generators]))
    transformed_generators = pca.transform(generators)
    transformed_generator_norms = np.linalg.norm(transformed_generators, axis=1, ord=2)
    # from largest to lowest norm
    sorted_indices = transformed_generator_norms.argsort()[::-1].astype(int)
    sorted_transformed_generators = transformed_generators[sorted_indices]
    sorted_monomials = [monomials[idx] for idx in sorted_indices]
    new_transformed_generators = np.concatenate([sorted_transformed_generators[:budget], 
                                                 np.diag(np.sum(np.abs(sorted_transformed_generators[budget:]), 
                                                                axis=0))])
    new_generators = pca.inverse_transform(new_transformed_generators)
    new_monomials = sorted_monomials[:budget] + [create_symbol() for _ in range(len(generators[0]))]
    
    processed_expr_ls = center
    for monomial_id in range(len(new_monomials)):
        processed_expr_ls += sympy.Matrix(new_generators[monomial_id])*new_monomials[monomial_id]
    
    return processed_expr_ls


def get_vertices(affset):
    l = len(affset)
    distinct_symbols = set()
    for expr in affset:
        if not(isinstance(expr, sympy.Expr)):
            assert isinstance(expr, int) or isinstance(expr, float)
        else:
            if distinct_symbols:
                distinct_symbols = distinct_symbols.union(expr.free_symbols)
            else:
                distinct_symbols = expr.free_symbols
    distinct_symbols = list(distinct_symbols)
    # print(distinct_symbols)
    combs = [list(zip(distinct_symbols,list(l))) for l in list(itertools.product([-1, 1], repeat=len(distinct_symbols)))]
    res = set()
    for assignment in combs:
        res.add(tuple([expr.subs(assignment) for expr in affset]))
    return(res)


# take a list of expressions as input, output the list of monomials and generator vectors,
def get_generators(expr_ls):
    monomials = dict()
    for expr_id, expr in enumerate(expr_ls):
        if not(isinstance(expr, sympy.Expr)) or not(expr.free_symbols):
            continue
        expr = expr.expand()
        p = sympy.Poly(expr)
        monomials_in_expr = [sympy.prod(x**k for x, k in zip(p.gens, mon)) 
                             for mon in p.monoms() if sum(mon) >= 1]
        for monomial in monomials_in_expr:
            coef = float(p.coeff_monomial(monomial))
            if monomial in monomials:
                if len(monomials[monomial]) < expr_id:
                    monomials[monomial] = monomials[monomial] + [0 for _ in range(expr_id-len(monomials[monomial]))]
                monomials[monomial].append(coef)
            else:
                monomials[monomial] = [0 for _ in range(expr_id)] + [coef]

    for monomial in monomials:
        if len(monomials[monomial]) < len(expr_ls):
            monomials[monomial] = monomials[monomial] + [0 for _ in range(len(expr_ls)-len(monomials[monomial]))]
    
    return monomials


def plot_conretiztion(affset, alpha = 0.5, color='red', budget=-1):
    if budget > -1:
        affset = merge_small_components_pca(affset, budget=budget)
    pts = np.array(list(map(list, get_vertices(affset))))
    hull = ConvexHull(pts)
    plt.fill(pts[hull.vertices,0], pts[hull.vertices,1],color,alpha=alpha)

In [51]:
from sklearn.preprocessing import StandardScaler

def inject_ranges(X, y, uncertain_attr, uncertain_num, uncertain_radius_pct=None, 
                  uncertain_radius=None, seed=42):
    global symbol_id
    symbol_id = -1
    
    X_extended = np.append(np.ones((len(X), 1)), X, axis=1)
    ss = StandardScaler()
    X_extended[:, 1:] = ss.fit_transform(X_extended[:, 1:])
    X_extended_symb = sympy.Matrix(X_extended)
    
    if not(uncertain_attr=='y'):
        uncertain_attr_idx = X.columns.to_list().index(uncertain_attr) + 1
        if not(uncertain_radius):
            uncertain_radius = uncertain_radius_pct*(np.max(X_extended[:, uncertain_attr_idx])-\
                                                     np.min(X_extended[:, uncertain_attr_idx]))
    else:
        if not(uncertain_radius):
            uncertain_radius = uncertain_radius_pct*(y_train.max()-y_train.min())[0]
    
    np.random.seed(seed)
    uncertain_indices = np.random.choice(range(len(y)), uncertain_num, replace=False)
    y_symb = sympy.Matrix(y)
    symbols_in_data = set()
    for uncertain_idx in uncertain_indices:
        new_symb = create_symbol()
        symbols_in_data.add(new_symb)
        if uncertain_attr=='y':
            y_symb[uncertain_idx] = y_symb[uncertain_idx] + uncertain_radius*new_symb
        else:
            X_extended_symb[uncertain_idx, uncertain_attr_idx] = X_extended_symb[uncertain_idx, uncertain_attr_idx] + uncertain_radius*new_symb
    return X_extended_symb, y_symb, symbols_in_data, ss


def sample_data_from_ranges(X, y, seed=42):
    all_free_symbols = X.free_symbols.union(y.free_symbols)
    subs_dict = dict()
    np.random.seed(seed)
    for symb in all_free_symbols:
        subs_dict[symb] = (np.random.uniform()-.5)*2
    return X.subs(subs_dict), y.subs(subs_dict)



Inject symbolic expressions (with uncertainty_radius=0.01) into all columns of X_calib:

Inject symbolic expressions (with uncertainty_radius=0.01) into all columns of X_calib:

In [1]:
# Inject symbolic expressions into all columns of X_calib.
# We inject into each column using the inject_ranges function.
# We use uncertain_num = len(X_calib) so that every row in the column gets an injection.
# First, inject for the first column and then update the remaining columns.

# Inject for the first column
first_col = X_calib.columns[0]
X_calib_symb, y_calib_symb, injected_symbols, scaler_used = inject_ranges(
    X_calib, y_calib, uncertain_attr=first_col, uncertain_num=len(X_calib),
    uncertain_radius_pct=None, uncertain_radius=0.01, seed=RANDOM_STATE
)

all_injected_symbols = injected_symbols.copy()

# Inject for the remaining columns and update the corresponding column entries
for col in X_calib.columns[1:]:
    X_tmp, _, symbs_tmp, _ = inject_ranges(
        X_calib, y_calib, uncertain_attr=col, uncertain_num=len(X_calib),
        uncertain_radius_pct=None, uncertain_radius=0.01, seed=RANDOM_STATE
    )
    col_idx = X_calib.columns.to_list().index(col) + 1  # +1 due to prepended ones column
    for i in range(len(X_calib)):
        X_calib_symb[i, col_idx] = X_tmp[i, col_idx]
    all_injected_symbols = all_injected_symbols.union(symbs_tmp)
    
# (Optional) Save all injected symbols into a dict for later reference
y_symb_dict['X_calib'] = all_injected_symbols

# Display a snippet of the symbolic matrix
X_calib_symb

NameError: name 'X_calib' is not defined